##코로나 관련 트윗 분석
1. 2020년 5월14일-16일 동안 생성된 '코로나' 키워드를 가진 트윗 10,000개 수집(크롤링)
2. 전처리 (불용어 제거)
3. FastText로 워드임베딩 생성 (skipgram, minCount=10)
4. Nearest neigbor 출력
5. 벡터공간 시각화

In [0]:
# GetOldTweet3 라이브러리로 트윗 크롤링
try:
    import GetOldTweets3 as got
except:
    !pip install GetOldTweets3
    import GetOldTweets3 as got

In [5]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

In [10]:
#1. 트윗 크롤링
#2020년5월14-16일간 생성된 '코로나' 키워드를 가진 트윗을 크롤링
#맥스트윗은 10,000개로 제한. HTTP 429 “Too Many Requests” 에러 방지
tweetCriteria = got.manager.TweetCriteria().setQuerySearch('코로나')\
                                           .setSince("2020-05-14")\
                                           .setUntil("2020-05-16")\
                                           .setMaxTweets(10000)
tweet = got.manager.TweetManager.getTweets(tweetCriteria)

tweets=[]
for cnt, tw in enumerate(tweet):
  tweets.append(tw.text)
  if cnt%100==0:
    print(tw.text)
  
print("total %d texts printed" %len(tweet))

코로나 때문에 촛불시위도 못할거라 생각 하고 막지르는 만주당...어디 까지ㅠ가나 보자...
강남 레깅스 셔츠룸 하실장 010=8264=4901 http://gangnamleggings01082644901.business.site #선릉셔츠룸 #강남가라오케 #풀싸롱 #강남셔츠룸 #강남셔츠룸x1x8264x4901 #서초셔츠룸 #강남룸 #슈퍼스타셔츠룸 #코로나 #코로나19_예방
구리시태극기달기운동본부, 코로나19 지역경제활성화 동참 
＂살아야 하니까＂…코로나 공포에도 '50대 가장'은 뛰었다 (출처 : MBC | 네이버 뉴스) http://naver.me/Fn6NGewv
[인뉴스TV] 문 대통령, '스타트업, 성장하기까지 죽음의 계곡 극복해야' 코로나 시대 구상 밝힌 이후 첫 경제행보, 스타트업 대표들과 위기 극복 간담회 
“우리 함께 코로나19 극복해요!” - 시사문경 
코로나19 방역에 힘쓰는 신천지 대전교회 https://loveismeet.tistory.com/675 #신천지 #신천지대전교회 #코로나19
"혈액형 A형인 사람이 코로나19 가장 많이 걸려..AB형은 드물어" | 다음 뉴스 
마스크가 굿즈? 개띠용했는데 코로나때문에 일본굿즈문화에 마스크가 포함되어가고있다니.... 
코로나땜에 집에만 있다보니 너무 심심해서 새로 만든거예요ㅋㅋㅋㅋ 저도 더 많이 대화해보고 싶어요!
ㅋㅋㅋ언제보내드리죠 ㅠ 아시발근데저 5분거리대학에서 코로나터져서 ㅡㅡ;;
코로나가 지나간다고 한들 내 삶에 변화가 크게 있진 않을거 같다. 여행이 가고싶긴한데 지금은 피곤함이랑 귀찮음이 먼저가 되어버렸다. 돈도 없고. 혼자서 노는 일도 예전만큼 재밌지도 않고. 그냥저냥하게 이렇게만 살아도 되는걸까라는 생각을 근래에 자주한다. 이렇게 반복적인 평범한 일상이
 -♡~코로나 행사이벤트~♡- 《 술집초.집초.길초.모초.다 가능합니다.》 오빠가간다 어디든 /o!o.6898.8OO5/o!o.6898.8OO5 #일탈ㄱㅖ #일탈 일탈 #사당호빠 #방배호빠 #방배동호빠 #서초호빠 #서초동호빠 #이수호빠 

In [16]:
#df=pd.read_csv("output_got.csv")
#tweet.to_csv('/content/drive/My Drive/Colab Notebooks/NLP/tweet_crawling/tweet_corpus.csv', encoding='utf-8-sig')
#크롤링한 GetOldTweets3 타입 모델을 처리가능한 리스트로 변환
total=[]
for tw in tweet:
  total.append(tw.text)
print(total[0])

코로나 때문에 촛불시위도 못할거라 생각 하고 막지르는 만주당...어디 까지ㅠ가나 보자...


In [0]:
#원본 Corpus를 CSV파일로 저장
df = pd.DataFrame(total)
df.to_csv('/content/drive/My Drive/Colab Notebooks/NLP/tweet_crawling/tweet_corpus.csv', encoding='utf-8-sig', index=False)

In [18]:
#2. 한국어 전처리 - 코엔엘파이 라이브러리 사용
!pip3 install konlpy    # Python 3.x

In [0]:
#불용어 제거
from konlpy.tag import Kkma
from konlpy.utils import pprint
kkma = Kkma()
pprint(kkma.nouns(total[3]))
for i in range(len(total)):
  total[i]=" ".join(kkma.nouns(total[i]))

print(total[3])

['세계', '세계속', '속', '방송', '시간', '코로나', '거스', '하나']


In [0]:
#전처리된 Corpus를 CSV파일로 저장
df = pd.DataFrame(total)
df.to_csv('/content/drive/My Drive/Colab Notebooks/NLP/tweet_crawling/tweet_corpus_pp.csv', encoding='utf-8-sig', index=False)

In [0]:
#Fasttext로 워드벡터생성
!pip install fasttext

import fasttext
# Skipgram model :
model = fasttext.train_unsupervised('/content/drive/My Drive/Colab Notebooks/NLP/tweet_crawling/tweet_corpus_pp.csv', model='skipgram', minCount=5)

# or, cbow model :
#model = fasttext.train_unsupervised('data.txt', model='cbow')

In [73]:
print(model.words)   # list of words in dictionary
print(model['코로나']) # get the vector of the word 'king'

['</s>', '코로나', '19', '코로나19', '때문', '거', '뉴스', '일', '나', '수', '1', '코', '때', '5', '사람', '2', '월', '3', '데', '진자', '우리', '시', '명', '전', '코로나때문', '집', '중', '안', '출처', '다음', '분', '생각', '감염', '이태원', '내', '저', 'ㅠ', '진', '오늘', '마스크', '클럽', '조심', '만', '말', '네이버', '대', '년', '한국', '발', '땜', '4', '지금', '검사', '6', '바이러스', '여', '서울', '사태', '확산', '터', '날', '개', '라', '9', '뭐', '이', '15', '등', '시간', '이번', '요즘', '님', '미국', '구', '바', '관련', '원', '방역', '8', '일본', '차', '미', '후', '주', '신천지', '감사', '학교', '방', '0', '중국', '듯', '상황', '이후', '5월', '진짜', '7', '발생', '건강', '신종', '세계', '20', '문', '기', '마음', '가능', '애', '달', '대응', '극복', '번', '오', '경제', '시대', '해', '이상', '나라', '돈', '거리', '10', '""', '디', '위', '지역', '다음뉴스', '하', '건', '이것', '취소', '정부', '하나', '노래방', '코로나땜', '환자', '간', '연기', '올해', '하루', '15일', '국민', '지원', '세', '카페', '친구', '백신', '사랑', '걱정', '곳', '씨', '2020', '김', '내가', 'ㅅ', '치료', '교회', '국내', 'ㅇ', '마', '비', '지', '줄', '정도', '속', '새끼', '현황', '일보', '조', '사회', '나도', '병원', '리', '격리', '직원', 'ㅂ', '안전', '의료', '당', '

In [78]:
model.get_nearest_neighbors("방역")

[(0.9318748116493225, '방역당국'),
 (0.9276729822158813, '방역수칙'),
 (0.924677848815918, '역'),
 (0.9180301427841187, '수칙'),
 (0.898414671421051, '사회'),
 (0.8967414498329163, '당국'),
 (0.8960665464401245, '시설'),
 (0.8853254914283752, '생활방역'),
 (0.8812808990478516, '지역사회'),
 (0.8785784244537354, '예방수칙')]

In [82]:
model.get_word_id('한국')

47

In [0]:
import re
import nltk

from gensim.models import word2vec

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.words:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [3]:
tsne_plot(model)

NameError: ignored